<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#工具包-&amp;-数据导入" data-toc-modified-id="工具包-&amp;-数据导入-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>工具包 &amp; 数据导入</a></span><ul class="toc-item"><li><span><a href="#开源工具包导入" data-toc-modified-id="开源工具包导入-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>开源工具包导入</a></span></li><li><span><a href="#自定义工具包导入" data-toc-modified-id="自定义工具包导入-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>自定义工具包导入</a></span></li><li><span><a href="#数据导入" data-toc-modified-id="数据导入-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>数据导入</a></span><ul class="toc-item"><li><span><a href="#原始数据读取" data-toc-modified-id="原始数据读取-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>原始数据读取</a></span></li><li><span><a href="#降低数据内存" data-toc-modified-id="降低数据内存-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>降低数据内存</a></span></li></ul></li></ul></li><li><span><a href="#特征工程" data-toc-modified-id="特征工程-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>特征工程</a></span><ul class="toc-item"><li><span><a href="#所有object特征进行labelencoder" data-toc-modified-id="所有object特征进行labelencoder-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>所有object特征进行labelencoder</a></span></li><li><span><a href="#frequency编码" data-toc-modified-id="frequency编码-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>frequency编码</a></span></li><li><span><a href="#数据合并" data-toc-modified-id="数据合并-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>数据合并</a></span></li></ul></li><li><span><a href="#模型训练&amp;测试" data-toc-modified-id="模型训练&amp;测试-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>模型训练&amp;测试</a></span><ul class="toc-item"><li><span><a href="#全量LGB" data-toc-modified-id="全量LGB-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>全量LGB</a></span></li><li><span><a href="#模型训练" data-toc-modified-id="模型训练-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>模型训练</a></span></li><li><span><a href="#模型测试" data-toc-modified-id="模型测试-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>模型测试</a></span></li><li><span><a href="#实验结果(LB:0.69397,-PB：0.63574-868/2426)" data-toc-modified-id="实验结果(LB:0.69397,-PB：0.63574-868/2426)-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>实验结果(LB:0.69397, PB：0.63574 868/2426)</a></span></li></ul></li><li><span><a href="#小结" data-toc-modified-id="小结-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>小结</a></span></li></ul></div>

# 工具包 & 数据导入
## 开源工具包导入

In [1]:
## 数据工具包
import pandas as pd
import numpy as np
np.random.seed(42)

from tqdm import tqdm 
from tqdm import tqdm_notebook
## 字符串处理工具包

import string
import re
import gensim
from collections import Counter
import pickle
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from keras.preprocessing import text, sequence 

import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
import lightgbm as lgb
from functools import partial
import joblib

import os 
import gc
from scipy.sparse import vstack  
import time 
import seaborn as sns
%matplotlib inline

/opt/conda/envs/zjpy36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 自定义工具包导入
- 该工具包主要用于降低数据的内存消耗,但是在使用时需要注意精度溢出的问题


In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm 
class _Data_Preprocess:
    def __init__(self):
        self.int8_max =  np.iinfo(np.int8).max
        self.int8_min =  np.iinfo(np.int8).min
        
        self.int16_max =  np.iinfo(np.int16).max
        self.int16_min =  np.iinfo(np.int16).min
      
        self.int32_max =  np.iinfo(np.int32).max
        self.int32_min =  np.iinfo(np.int32).min
      
        self.int64_max =  np.iinfo(np.int64).max
        self.int64_min =  np.iinfo(np.int64).min
        
        self.float16_max =  np.finfo(np.float16).max
        self.float16_min =  np.finfo(np.float16).min
        
        self.float32_max =  np.finfo(np.float32).max
        self.float32_min =  np.finfo(np.float32).min
      
        self.float64_max =  np.finfo(np.float64).max
        self.float64_min =  np.finfo(np.float64).min
         
    
    '''
    function: _get_type(self,min_val, max_val, types)
    
       get the correct types that our columns can trans to
    
    '''
    def _get_type(self,min_val, max_val, types):
        if types == 'int':
            if max_val <= self.int8_max and min_val >= self.int8_min:
                return np.int8
            elif max_val <= self.int16_max  <= max_val and min_val >= self.int16_min:
                return np.int16
            elif max_val <= self.int32_max and min_val >= self.int32_min:
                return np.int32
            return None
            
        elif types == 'float':
            if max_val <=  self.float16_max and min_val >= self.float16_min :
                return np.float16
            if max_val <=  self.float32_max and min_val >= self.float32_min :
                return np.float32
            if max_val <=  self.float64_max and min_val >= self.float64_min:
                return np.float64 
            return None
    '''
    
    function: _memory_process(self,df) 
       column data types trans, to save more memory
    '''
    def _memory_process(self,df):
        init_memory = df.memory_usage().sum() / 1024**2 / 1024
        print('Original data occupies {} GB memory.'.format(init_memory))
        df_cols = df.columns
        
        for col in tqdm(df_cols):
            try:
                if 'float' in str(df[col].dtypes): 
                    max_val = df[col].max()
                    min_val = df[col].min()
                    trans_types = self._get_type(min_val,max_val,'float')
                    if trans_types is not None: 
                        df[col] = df[col].astype(trans_types)
                elif 'int' in str(df[col].dtypes):  
                    max_val = df[col].max()
                    min_val = df[col].min()
                    trans_types = self._get_type(min_val,max_val,'int')
                    if trans_types is not None:  
                        df[col] = df[col].astype(trans_types) 
            except:
                print(' Can not do any process for column, {}.'.format(col))
        afterprocess_memory = df.memory_usage().sum() / 1024**2 / 1024
        print('After processing, the data occupies {} GB memory.'.format(afterprocess_memory))
        return df

 ## 数据导入
 ### 原始数据读取

In [5]:
path = '/data/Data_JieZhang/MicroMalware/'

In [6]:
%%time
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')

CPU times: user 2min 47s, sys: 12.8 s, total: 2min 59s
Wall time: 3min 8s


In [7]:
test['HasDetections'] = -1

In [8]:
train_test = pd.concat([train,test],ignore_index = True)

### 降低数据内存
- 因为数据内存较大,如果机器内存较小,可以考虑先对数据的内存进行处理,当然还有一种更加方便的策略,就是在读取数据的时候就指定每列特征是int还是float型,只是那样得比较细心,各位可以按照自己的喜好进行处理。

In [9]:
data_preprocess = _Data_Preprocess()

In [10]:
%%time
train_test = data_preprocess._memory_process(train_test) 

  0%|          | 0/83 [00:00<?, ?it/s]

Original data occupies 10.373466446995735 GB memory.


100%|██████████| 83/83 [00:16<00:00,  2.94it/s]

After processing, the data occupies 5.749150112271309 GB memory.
CPU times: user 14.9 s, sys: 1.89 s, total: 16.8 s
Wall time: 16.7 s


# 特征工程
## 所有object特征进行labelencoder
- 为了不损失object特征的信息,我们对所有的object特征进行labelencoder,从而将object特征转化为整型,方便用于模型的训练&测试.


In [12]:
%%time
object_cols = list(train_test.dtypes[train_test.dtypes == 'object'].index)
object_cols.remove('MachineIdentifier') 

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.44 ms


In [16]:
lbencoding_fea = pd.DataFrame()
lbencoding_fea['MachineIdentifier'] = train_test['MachineIdentifier'].values
for col in tqdm_notebook(object_cols):
    lbencoding_fea[col + '_labelcoder'] = LabelEncoder().fit_transform(train_test[col].astype(str).fillna('NAN').values)

HBox(children=(IntProgress(value=0, max=29), HTML(value='')))

In [17]:
%%time
for col in tqdm(train_test.columns):
    if col!='MachineIdentifier' and col + '_labelcoder' in lbencoding_fea.columns:
        train_test[col] = lbencoding_fea[col+ '_labelcoder' ].values

100%|██████████| 83/83 [00:06<00:00, 12.30it/s]

CPU times: user 5.19 s, sys: 1.59 s, total: 6.78 s
Wall time: 6.75 s


- 降低数据的内存使用

In [22]:
%%time
lbencoding_fea = data_preprocess._memory_process(lbencoding_fea) 

  0%|          | 0/30 [00:00<?, ?it/s]

Original data occupies 3.749445751309395 GB memory.


100%|██████████| 30/30 [00:07<00:00,  3.79it/s]

After processing, the data occupies 0.7186438292264938 GB memory.
CPU times: user 6.05 s, sys: 1.9 s, total: 7.94 s
Wall time: 7.91 s


## frequency编码
- <font color=red>frequency编码在此处可以认为是一种流行度的编码,如果一个东西出现的越多,比如浏览器中Chrome出现的非常多,也就是说安装Chrome的用户非常多,那么通过frequency编码就可以很好的反映出Chrome的流行度使用量较高的信息。同样地,对于其他的版本信息也是一样的。</font>
- 在微软的比赛中,frequency编码反映的另外一种信息是,如果某个产品越加流行,那么该产品的利益价值就越高,黑客可能就更加喜欢攻击此类产品


In [18]:
train_test_nunique = train_test.nunique()
A_cnt_features = [col for col in train_test_nunique.index if train_test_nunique.loc[col] > 5 and col!='MachineIdentifier']
len(A_cnt_features)

55

In [19]:
A_cnt_features

['ProductName',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'RtpStateBitfield',
 'DefaultBrowsersIdentifier',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'AVProductsEnabled',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'OsVer',
 'OsBuild',
 'OsSuite',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'IeVerIdentifier',
 'SmartScreen',
 'UacLuaenable',
 'Census_MDC2FormFactor',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_PrimaryDiskTotalCapacity',
 'Census_SystemVolumeTotalCapacity',
 'Census_TotalPhysicalRAM',
 'Census_ChassisTypeName',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_PowerPlatformRoleName',
 'Census_InternalBatteryType

In [20]:
frequency_fea = pd.DataFrame()
frequency_fea['MachineIdentifier'] = train_test['MachineIdentifier'].values

for col in tqdm_notebook(A_cnt_features):
    train_test[col]             = train_test[col].fillna(-999)
    frequency_fea[col + '_cnt'] = train_test[col].map(train_test[col].value_counts())

HBox(children=(IntProgress(value=0, max=55), HTML(value='')))

- 降低数据的内存使用

In [21]:
%%time
frequency_fea = data_preprocess._memory_process(frequency_fea) 

  0%|          | 0/56 [00:00<?, ?it/s]

Original data occupies 6.998965337872505 GB memory.


100%|██████████| 56/56 [00:13<00:00,  3.46it/s]

After processing, the data occupies 3.5619734674692154 GB memory.
CPU times: user 12 s, sys: 1.9 s, total: 13.9 s
Wall time: 13.8 s


## 数据合并

In [23]:
%%time
merge_cols = ['MachineIdentifier','IsBeta', 'RtpStateBitfield', 'IsSxsPassiveMode','DefaultBrowsersIdentifier', 'AVProductStatesIdentifier',
       'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm', 'CountryIdentifier', 'CityIdentifier', 'OrganizationIdentifier',
       'GeoNameIdentifier', 'LocaleEnglishNameIdentifier', 'OsBuild','OsSuite', 'IsProtected', 'AutoSampleOptIn', 'SMode', 'IeVerIdentifier',
       'Firewall', 'UacLuaenable', 'Census_OEMNameIdentifier','Census_OEMModelIdentifier', 'Census_ProcessorCoreCount',
       'Census_ProcessorManufacturerIdentifier','Census_ProcessorModelIdentifier', 'Census_PrimaryDiskTotalCapacity',
       'Census_SystemVolumeTotalCapacity', 'Census_HasOpticalDiskDrive',
       'Census_TotalPhysicalRAM','Census_InternalPrimaryDiagonalDisplaySizeInInches','Census_InternalPrimaryDisplayResolutionHorizontal',
       'Census_InternalPrimaryDisplayResolutionVertical','Census_InternalBatteryNumberOfCharges', 'Census_OSBuildNumber',
       'Census_OSBuildRevision', 'Census_OSInstallLanguageIdentifier','Census_OSUILocaleIdentifier', 'Census_IsPortableOperatingSystem',
       'Census_IsFlightingInternal', 'Census_IsFlightsDisabled','Census_ThresholdOptIn', 'Census_FirmwareManufacturerIdentifier',
       'Census_FirmwareVersionIdentifier', 'Census_IsSecureBootEnabled', 'Census_IsWIMBootEnabled', 'Census_IsVirtualDevice',
       'Census_IsTouchEnabled', 'Census_IsPenCapable','Census_IsAlwaysOnAlwaysConnectedCapable', 'Wdft_IsGamer', 'Wdft_RegionIdentifier', 'HasDetections']
train_test_data = train_test[merge_cols].merge(lbencoding_fea, on ='MachineIdentifier', how='left')
train_test_data = train_test_data.merge(frequency_fea, on ='MachineIdentifier', how='left')

CPU times: user 1min 12s, sys: 14.2 s, total: 1min 26s
Wall time: 1min 26s


In [24]:
train_set = train_test_data.loc[train_test_data['HasDetections'].isin([0,1]) == True]
test_set  = train_test_data.loc[train_test_data['HasDetections'].isin([0,1]) == False]

# 模型训练&测试

- 因为此次数据的量较大,采用5折的LGB训练虽然可以取得不错的成绩,但是迭代的过程较慢,而且数据量非常的大,不是一个很好的选择,当然如果大家希望在比赛过程中获得更好的成绩,建议大家采用N-fold的训练测试的方法。

## 全量LGB

In [ ]:
def _single_lgbmodel(df, df_label):
    importances = pd.DataFrame()
    lgb_params = {'objective':'binary', "boosting": "gbdt", 'learning_rate': 0.03, 'max_depth': -1, 
     "feature_fraction": 0.8, "bagging_freq": 1, "bagging_fraction": 0.8 , "bagging_seed": 11, 'n_estimators': 2500,
     "metric": 'auc', "lambda_l1": 0.1, 'num_leaves': 256, 'min_data_in_leaf': 50, "verbose": 1, "num_threads": 50}
      
    
    model = lgb.LGBMClassifier(**lgb_params)
    model.fit(df, df_label, eval_set=[(df, df_label)], verbose = 250, eval_metric ='auc')     

    imp_df = pd.DataFrame()
    imp_df['feature'] = df.columns
    imp_df['gain']    = model.feature_importances_
        
    gc.collect()
    return model,imp_df 

## 模型训练

In [25]:
train_cols = [col for col in train_test_data.columns if col not in ['MachineIdentifier','HasDetections','isval']]
train_label = 'HasDetections'

In [26]:
%%time
model, model_imp = _single_lgbmodel(train_set[train_cols], train_set[train_label].values)

[250]	training's auc: 0.734275
[500]	training's auc: 0.743315
[750]	training's auc: 0.747874
[1000]	training's auc: 0.751305
[1250]	training's auc: 0.7543
[1500]	training's auc: 0.75718
[1750]	training's auc: 0.75987
[2000]	training's auc: 0.76242
[2250]	training's auc: 0.764876
[2500]	training's auc: 0.767355
CPU times: user 2d 15h 27s, sys: 1h 53min 58s, total: 2d 16h 54min 25s
Wall time: 1h 58min 22s


## 模型测试

In [27]:
pred = model.predict_proba(test_set[train_cols])[:,1]
test_set['HasDetections'] = pred
test_set['HasDetections'] = test_set['HasDetections']
test_set[['MachineIdentifier','HasDetections']].to_csv('baseline_labelencode_frequency.csv',index = None)

## 实验结果(LB:0.69397, PB：0.63574 868/2426)
![](./pic/baseline_score.png)

# 小结

在本篇文章中,我们给出了两种处理类别特征的常用方案：

1. labelencoder;
2. frequency encoder.

最后为了方便我们采用全量lgb对我们的数据进行训练&测试,在下一节的内容中,我们会在本篇文章的基础上进一步提升我们模型的性能。

